In [1]:
import os
import pandas as pd
import pickle as pkl
import glob
from process_jtwc import *
from itertools import product
import pymongo
import requests, zipfile, io
import csv
import pdb

# JTWC

From https://www.metoc.navy.mil/jtwc/jtwc.html?best-tracks

# Download files for certain years and regions.

In [3]:
#download JTWC data for certain years

years = [ x for x in range(2000, 2001+1)]
regions = ['bio', 'bsh', 'bwp']
def get_tc_by_year_region(year=2018, region='bio'):
    url = make_url(year, region)
    resp = requests.get(url)
    if not resp.status_code // 100 == 2:
        return "Error: Unexpected response {}".format(resp)
    z = zipfile.ZipFile(io.BytesIO(resp.content))
    file_path = make_file_path(year, region)
    z.extractall(file_path)

def make_file_path(year, region, base='/storage/hurricane'):
    filePath = os.path.join(base, region, str(year))
    if not os.path.exists(filePath):
        os.makedirs(filePath)
    return filePath

def make_url(year=2018, region='bio'):
    return f'https://www.metoc.navy.mil/jtwc/products/best-tracks/{year}/{year}s-{region}/{region}{year}.zip'

def download_jtwc_files(years, regions, base='/storage/hurricane'):
    for year, region in product(years, regions):
        make_file_path(year, region, base)
        get_tc_by_year_region(year, region)

In [3]:
download_jtwc_files(years, regions)

## Convert files into a dataframe

In [2]:
df_lst = []
dr = glob.glob('/storage/hurricane/*/*/*.dat') # location of files taken from website.
ulist = []
for fn in dr:
#     print(fn)
    raw = pd.read_table(fn, header=None, delimiter=',', usecols=range(11))
    df_raw = convert_df(raw)
    ulist = ulist + df_raw.ID.unique().tolist()
    df_lst = df_lst + df_raw.to_dict(orient='records')
df = pd.DataFrame(df_lst)
df['source'] = 'JTWC'
df = df.applymap(lambda x: x.replace(' ', '') if isinstance(x, str) else x)

In [3]:
df = df.rename({"ID":'_id', "LONG": 'lon', 'press': 'pres', 'SEASON':'year'}, axis=1)
df.columns = [col.lower() for col in df.columns]
df.year = df.year.astype(np.int64)
df.time = df.time.astype(np.int64)
df.lat = df.lat.astype(np.float64)
df.lon = df.lon.astype(np.float64)
df['geoLocation'] = [ {"type": "point", "coordinates": [lng, lat]} for lng, lat in df[['lon', 'lat']].values]

In [4]:
df.head()

,_id,name,date,time,l,class,lat,lon,wind,press,year,num,timestamp,source,geoLocation
0,IO012011,UNNAMED,2011-06-09,9000,,DB,18.1,69.9,20,1007,2011,1,2011-06-09 06:00:00,JTWC,"{'type': 'point', 'coordinates': [69.9, 18.1]}"
1,IO012011,UNNAMED,2011-06-09,9100,,TD,18.3,70.3,25,1004,2011,1,2011-06-09 12:00:00,JTWC,"{'type': 'point', 'coordinates': [70.3, 18.3]}"
2,IO012011,UNNAMED,2011-06-09,9100,,TD,18.6,70.6,25,1004,2011,1,2011-06-09 18:00:00,JTWC,"{'type': 'point', 'coordinates': [70.6, 18.6]}"
3,IO012011,UNNAMED,2011-06-10,0,,TD,19.0,70.7,25,1004,2011,1,2011-06-10 00:00:00,JTWC,"{'type': 'point', 'coordinates': [70.7, 19.0]}"
4,IO012011,UNNAMED,2011-06-10,0,,TD,19.1,71.1,25,1004,2011,1,2011-06-10 06:00:00,JTWC,"{'type': 'point', 'coordinates': [71.1, 19.1]}"


In [5]:
df.lat.min()

-41.0

In [6]:
def is_empty(x):
    if isinstance(x, str):
        return x == ''
    elif isinstance(x, float):
        return np.isnan(x)
    else:
        return False

clean_dict_keys = lambda my_dict: list(filter(lambda k: not is_empty(my_dict[k]), my_dict))
def clean_dict(my_dict):
    new_dict = dict()
    keys = clean_dict_keys(my_dict)
    for key in keys:
        new_dict[key] = my_dict[key]
    return new_dict

def make_docs(df):
    docs = []
    keys = ['_id', 'name', 'num', 'source']
    key_types = {'_id': str, 'name':str , 'num': int, 'source': str}
    cols = [col for col in df.columns if col not in keys]
    for _id, df_id in df.groupby(['_id']):
        df_id.shape
        doc = {}
        for key in keys:
            tpe = key_types[key]
            assert len(df_id[key].unique()) == 1, 'nondistinct id'
            doc[key] = df_id[key].astype(tpe).iloc[0]
            if key == 'num':
                doc[key] = int(doc[key])
        traj_data = df_id[cols].to_dict(orient='records')
        traj_data = [clean_dict(x) for x in traj_data]
        year = int(df_id.year.iloc[0])
        doc['year'] = year
        doc['startDate'] = df_id.timestamp.min()
        doc['endDate'] = df_id.timestamp.max()
        doc['traj_data'] = df_id[cols].to_dict(orient='records')
        doc['_id'] = doc['_id'] + '_' + doc['source']
        if doc['name'] == 'UNNAMED':
            del doc['name']
        docs.append(doc)
    docs = [clean_dict(x) for x in docs]
    return docs

In [8]:
docs = make_docs(df)

In [9]:
docs[0].keys()

dict_keys(['_id', 'num', 'source', 'year', 'startDate', 'endDate', 'traj_data'])

# HURDAT2
from https://www.nhc.noaa.gov/data/#hurdat

In [7]:
pacific_filename = '/storage/hurricane/hurdat2/pacific.csv'
atlantic_filename = '/storage/hurricane/hurdat2/atlantic.csv'
stormStartCh = ['EP', 'CP', 'AL']

def make_trop_cyc_list(filename):
    startIdx = []
    tcs = []
    with open(filename) as csvfile:
        spamreader = csv.reader(csvfile, delimiter=',')
        for idx, row in enumerate(spamreader):
            if len(row) == 0:
                continue
            if row[0][0:2] in stormStartCh:
                startIdx.append(idx)
                _id = row[0]
                name= row[1]
                num = row[2]
                storm = [_id, name, num]
            else:
                tc = storm + row
                tcs.append(tc[0:11])
    return tcs

pacific_tcs = make_trop_cyc_list(pacific_filename)
atlantic_tcs = make_trop_cyc_list(atlantic_filename)

In [8]:
cols = ['_id', 'name', 'num', 'date', 'time', 'l', 'class', 'lat', 'lon', 'wind', 'pres']
def convert_lat_lon(strL, postiveDir='N'):
    L = float(strL[:-1].replace(' ', ''))
    if not postiveDir in strL:
        L *= -1
    return L

def convert_time(time):
    hour = (int(time)/100) %12 
    return hour

def make_cyc_df(tcs):
    df = pd.DataFrame(tcs, columns=cols)
    df['year'] = df.date.apply(lambda x: int(x[0:4]))
    df = df[df['year'] >= 2000]
    df = df.dropna(axis=0, how='any', subset=['lat', 'lon'])
    df = df.applymap(lambda x: x.replace(' ', '') if isinstance(x, str) else x)
    df.lon = df.lon.apply(lambda lon: convert_lat_lon(lon, 'E')).astype(np.float64)
    df.lat = df.lat.apply(lambda lat: convert_lat_lon(lat, 'N')).astype(np.float64)
    df.pres = df.pres.astype(np.int64)
    df.pres = df.pres.replace(-999, np.nan)
    df.wind = df.wind.astype(np.int64)
    df.num = df.num.astype(np.int64)
    df.date = df.date.apply(lambda x: x[0:4] + '-' + x[4:6] + '-' + x[6:8])
    datetimes = df.date.values + ' ' + df.time.values
    df['timestamp'] = pd.to_datetime(datetimes, format='%Y-%m-%d %H%M')
    df['source'] = 'HURDAT2'
    df['geoLocation'] = [ {"type": "point", "coordinates": [lng, lat]} for lng, lat in df[['lon', 'lat']].values]
    df.time = df.time.astype(np.int64)
    return df

df_pacific = make_cyc_df(pacific_tcs)
df_atlantic = make_cyc_df(atlantic_tcs)

In [11]:
df_pacific.lat.min()

0.4

In [12]:
df_atlantic.lat.min()

7.5

In [9]:
df_pacific.head()

,_id,name,num,date,time,l,class,lat,lon,wind,pres,year,timestamp,source,geoLocation
18050,EP012000,ALETTA,23,2000-05-22,1200,,TD,13.3,-99.2,25,1005.0,2000,2000-05-22 12:00:00,HURDAT2,"{'type': 'point', 'coordinates': [-99.2, 13.3]}"
18051,EP012000,ALETTA,23,2000-05-22,1800,,TD,13.9,-100.2,30,1004.0,2000,2000-05-22 18:00:00,HURDAT2,"{'type': 'point', 'coordinates': [-100.2, 13.9]}"
18052,EP012000,ALETTA,23,2000-05-23,0,,TD,14.3,-101.1,30,1003.0,2000,2000-05-23 00:00:00,HURDAT2,"{'type': 'point', 'coordinates': [-101.1, 14.3]}"
18053,EP012000,ALETTA,23,2000-05-23,600,,TS,14.5,-101.8,35,1002.0,2000,2000-05-23 06:00:00,HURDAT2,"{'type': 'point', 'coordinates': [-101.8, 14.5]}"
18054,EP012000,ALETTA,23,2000-05-23,1200,,TS,14.5,-102.5,40,1001.0,2000,2000-05-23 12:00:00,HURDAT2,"{'type': 'point', 'coordinates': [-102.5, 14.5]}"


In [10]:
hurdat2_docs = make_docs(df_pacific) + make_docs(df_atlantic)

In [9]:
df_lane = df_pacific[(df_pacific['name'] == 'LANE') & (df_pacific['year'] == 2018)]
df_lane.head()

,_id,name,num,date,time,l,class,lat,lon,wind,pres,year,timestamp,source,geoLocation
28126,EP142018,LANE,64,2018-08-13,1200,,LO,10.9,-114.7,20,1009.0,2018,2018-08-13 12:00:00,HURDAT2,"{'type': 'point', 'coordinates': [-114.7, 10.9]}"
28127,EP142018,LANE,64,2018-08-13,1800,,LO,11.0,-115.7,20,1009.0,2018,2018-08-13 18:00:00,HURDAT2,"{'type': 'point', 'coordinates': [-115.7, 11.0]}"
28128,EP142018,LANE,64,2018-08-14,0,,LO,11.1,-116.6,25,1009.0,2018,2018-08-14 00:00:00,HURDAT2,"{'type': 'point', 'coordinates': [-116.6, 11.1]}"
28129,EP142018,LANE,64,2018-08-14,600,,LO,11.1,-117.4,25,1009.0,2018,2018-08-14 06:00:00,HURDAT2,"{'type': 'point', 'coordinates': [-117.4, 11.1]}"
28130,EP142018,LANE,64,2018-08-14,1200,,LO,11.1,-118.4,25,1009.0,2018,2018-08-14 12:00:00,HURDAT2,"{'type': 'point', 'coordinates': [-118.4, 11.1]}"


In [12]:
test_doc = make_docs(df_lane)

# add docs to mongoDB database

In [14]:
def create_collection(dbName, collectionName, init_collection):
    dbUrl = 'mongodb://localhost:27017/'
    client = pymongo.MongoClient(dbUrl)
    db = client[dbName]
    coll = db[collectionName]
    coll = init_collection(coll)
    return coll

def init_tc_collection(coll):
    coll.create_index([('name', pymongo.DESCENDING)])
    coll.create_index([('startDate', pymongo.DESCENDING)])
    coll.create_index([('endDate', pymongo.DESCENDING)])
    coll.create_index([('startDate', pymongo.DESCENDING), ('endDate', pymongo.DESCENDING)])
    return coll

def init_tc_traj_collection(coll):
    return coll

In [15]:
#insert docs
dbName='argo'
collectionName = 'tc'
coll = create_collection(dbName, collectionName, init_tc_collection)
coll.drop()
coll.insert_many(docs)
coll.insert_many(hurdat2_docs)

In [16]:
dbName='argo-express-test'
collectionName = 'tc'
coll = create_collection(dbName, collectionName, init_tc_collection)
coll.drop()
coll.insert_many(test_doc)